In [ ]:
import pandas as pd

def load_finance_urls(excel_filename):
    df = pd.read_csv(excel_filename)
    
    return df['article_url'].tolist()

excel_filename = 'news_finance_final.csv'

finance_list = load_finance_urls(excel_filename)

In [ ]:
import pandas as pd
import requests
import json

article_url_list = []
content_list = []
comment_date_list = []
user_id_list = []


def scrape_url(url):
    data = {"article_url": [], "content": [], "comment_date": [], "user_id": []}
    try:
        # Extract user IDs from JSON data
        article_id = url.split('/')[-1]
        comment_api_url = f"https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=view_economy_m1&pool=cbox5&_cv=20240826125754&lang=ko&country=KR&objectId=news243,{article_id}&pageSize=100"
        custom_header = {
            "Referer": url,
            "User-Agent": "PostmanRuntime/7.41.2"
        }
        response = requests.get(comment_api_url, headers=custom_header)
        tester = response.text.split("_callback")[-1]
        json_string = tester.strip('()').replace('true', 'true'.lower())

        try:
            json_data = json.loads(json_string)
        except json.JSONDecodeError as e:
            print(f"JSONDecodeError: {e}")
            last_valid_position = json_string.rfind('}')
            if last_valid_position != -1:
                json_data = json.loads(json_string[:last_valid_position + 1])
            else:
                raise ValueError("유효한 JSON을 찾을 수 없습니다.")

        comments = json_data['result']['commentList']
        
        for comment in comments:
            data["article_url"].append(url)
            data["user_id"].append(comment.get("userIdNo", "No User ID"))
            data["content"].append(comment.get("contents", "No Content"))
            data["comment_date"].append(comment.get("regTime", "No Comment Date"))
        
    except Exception as e:
        print(f"Error occurred for {url}: {e}")
        data["article_url"].append(url)
        data["content"].append("Error")
        data["comment_date"].append("Error")
        data["user_id"].append("Error")
    return data

#url 리스트
urls = finance_list


for url in urls:
    result = scrape_url(url)
    article_url_list.extend(result["article_url"])
    content_list.extend(result["content"])
    comment_date_list.extend(result["comment_date"])
    user_id_list.extend(result["user_id"])

# 데이터 프레임 형성
comment_df = pd.DataFrame({
    "article_url": article_url_list,
    "content": content_list,
    "comment_date": comment_date_list,
    "user_id": user_id_list,
})

# 저장
comment_df.to_csv("finance_comments.csv", index=False, encoding='utf-8-sig')

print("Data scraping and saving to CSV completed successfully.")
